## import libraries

In [7]:
import pandas as pd
import numpy as np
import time
import datetime as datetime
import math
import requests
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, CrosshairTool, HoverTool, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.transform import cumsum
import yfinance as yf

In [17]:
from bokeh.io import output_notebook
output_notebook()

In [8]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
line_name = ["Reliance Industries", "ITC Limited","IRCTC Pvt Limited","TATA Motors","Infosys LTD"]

In [21]:
tickers= ['RELIANCE.NS','ITC.NS','IRCTC.NS','TATAMOTORS.NS','INFY.NS']
start= '2018-01-01'
end= '2022-09-30'

data= yf.download(tickers, start, end)['Adj Close']
data

[*********************100%***********************]  5 of 5 completed


,INFY.NS,IRCTC.NS,ITC.NS,RELIANCE.NS,TATAMOTORS.NS
Date,,,,,
2018-01-01 00:00:00+05:30,460.407837,NaN,218.016342,881.918213,424.450012
2018-01-02 00:00:00+05:30,458.692688,NaN,216.728027,883.275330,439.299988
2018-01-03 00:00:00+05:30,454.950836,NaN,217.060486,886.813660,433.899994
2018-01-04 00:00:00+05:30,452.322601,NaN,217.392960,892.145447,429.950012
2018-01-05 00:00:00+05:30,450.808044,NaN,218.515045,895.005188,431.600006
...,...,...,...,...,...
2022-09-23 00:00:00+05:30,1365.449951,685.799988,346.399994,2439.500000,423.100006
2022-09-26 00:00:00+05:30,1380.250000,664.500000,332.600006,2377.350098,397.500000
2022-09-27 00:00:00+05:30,1393.550049,676.650024,334.850006,2396.250000,398.799988


Tha user has to specify:

the list of symbols of stocks or the ETFs of interest (e.g. "TSLA" for Tesla, Inc.; "MSFT" for Microsoft Corporation; "VOO" for the Vanguard 500 Index Fund ETF). Recommended max number of entries: 10
a short description of each stocks or ETFs (in the same order as above)
the number of portolios she/he wants to simulate. Recommended number of portfolios to simulate > 5000

In [10]:
num_port = 1000

In [11]:
log_ret = np.log(data/data.shift(1))

cov_mat = log_ret.cov() * 252

all_wts = np.zeros((num_port, len(data.columns)))
port_returns = np.zeros((num_port))
port_risk = np.ones((num_port))
sharpe_ratio = np.zeros((num_port))

In [23]:
np.random.seed(42)

for i in range(num_port):

    # Portfolio weights
    wts = np.random.uniform(size=len(data.columns))
    wts = wts / np.sum(wts)
    all_wts[i, :] = wts

    # Portfolio Return
    port_ret = np.sum(log_ret.mean() * wts)
    port_ret = (port_ret + 1) ** 252 - 1
    port_returns[i] = port_ret

    # Portfolio Risk
    port_sd = np.sqrt(np.dot(wts.T, np.dot(cov_mat, wts)))
    port_risk[i] = port_sd

    # Portfolio Sharpe Ratio, assuming 0% Risk Free Rate
    sr = port_ret / port_sd
    sharpe_ratio[i] = sr

    # Save portfolios of interest (min var, max return and max SR)
    if sr >= max(sharpe_ratio[0:-1]):
        max_sr_ret = port_ret
        max_sr_risk = port_sd
        max_sr_w = wts
        max_sr = sr

    if port_ret >= max(port_returns[0:-1]):
        max_ret_ret = port_ret
        max_ret_risk = port_sd
        max_ret_w = wts

    if port_sd <= min(port_risk[0:-1]):
        min_var_ret = port_ret
        min_var_risk = port_sd
        min_var_w = wts
## PLOTS

# Modern Portfolio Theory: the efficient frontier
p = figure(plot_height=700, plot_width=770, title="Efficient frontier. Simulations: " + str(num_port),
           tools='box_zoom,wheel_zoom,reset', toolbar_location='above')
p.add_tools(CrosshairTool(line_alpha=1, line_color='lightgray', line_width=1))
p.add_tools(HoverTool(tooltips=None))
source = ColumnDataSource(data=dict(risk=port_risk, profit=port_returns))
p.circle(x='risk', y='profit', source=source, line_alpha=0, hover_color='navy', alpha=0.4, hover_alpha=1, size=8)
p.circle(min_var_risk, min_var_ret, color='tomato', legend='Portfolio with minimum variance', size=10)
p.circle(max_sr_risk, max_sr_ret, color='orangered', legend='Portfolio with max Sharpe ratio', size=12)
p.circle(max_ret_risk, max_ret_ret, color='firebrick', legend='Portfolio with max return', size=9)
p.legend.location = "top_left"
p.xaxis.axis_label = 'Volatility, or risk (standard deviation)'
p.yaxis.axis_label = 'Annual return'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

# Portfolio composition. Min variance, max SR, max return
def plot_portfolio_composition(tickers, weights, plot_name):

    x = dict()
    for i in range(len(tickers)):
        x[tickers[i]] = weights[i]

    color_list = [ "olive", "yellowgreen", "lime", "chartreuse", "springgreen", "lightgreen", "darkseagreen",
                   "seagreen", "green", "darkgreen"]

    plot_data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'stock'})
    plot_data['angle'] = plot_data['value'] / plot_data['value'].sum() * 2 * math.pi
    plot_data['color'] = color_list[0:len(weights)]
    p = figure(plot_height=250, plot_width=250, title=plot_name, toolbar_location=None,
                      tools="hover", tooltips="@stock: @value{%0.1f}", x_range=(-0.5, 1.0))
    p.wedge(x=0, y=1, radius=0.4, start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                   line_color="white", color='color', source=plot_data)
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_color = None
    return p

p_minvar = plot_portfolio_composition(tickers, min_var_w, "Portfolio with minimum variance")
p_maxsr = plot_portfolio_composition(tickers, max_sr_w, "Portfolio with max Sharpe ratio")
p_maxret = plot_portfolio_composition(tickers, max_ret_w, "Portfolio with max return")



# Stock prices over time
color_list = ["olive", "yellowgreen", "lime", "chartreuse", "springgreen", "lightgreen", "darkseagreen",
              "seagreen", "green", "darkgreen"]

p_time = figure(plot_height=450, plot_width=675, toolbar_location=None, tools="",
                title='Time series of stock prices in time. From ' + start + ' to ' + end)

i = 0
val_min_var = 0
val_max_shr = 0
val_max_ret = 0

for tick in tickers:
    p_time.line(range(0, len(data[tick])), list(data[tick]/(data[tick][0])),
                color=color_list[i], line_width=1, legend=line_name[i]) 

    val_max_shr += data[tick] / (data[tick][0]) * max_sr_w[i]

    i += 1

p_time.line(range(0, len(data[tickers[0]])), list(val_max_shr), legend="Portfolio with min SR",
            color='orangered', line_width=2.5)

p_time.legend.location = "top_left"
p_time.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
p_time.xaxis.axis_label = 'Trading day'
p_time.yaxis.axis_label = 'Return'


# Create dashboard and open new window to show results
layout = row([p, column([p_time, row([p_minvar, p_maxsr, p_maxret])])])
show(layout)